# The mirroring is quite computational expensive therefore the diagonal blocks get precomputed

In [1]:
import cloudpickle
from scipy.sparse import dok_matrix
from scipy.sparse import tril
import scipy

def load_dict(path,zeilen,spalten):
    if(spalten > zeilen):
        template = "block_{i}_{j}".format(i=spalten,j=zeilen)
    else:
        template = "block_{i}_{j}".format(i=zeilen,j=spalten)

    file_path = path + '\\' + template
    with open(file_path, 'rb+') as file:
        co_occurences = cloudpickle.load(file)
    
    return co_occurences


def load_co_occurence(path,zeilen,spalten):
    co_occurences = load_dict(path,zeilen,spalten)
    coocurrence = dok_matrix((20000,20000),dtype='d')
    
    coocurrence._update(co_occurences) # dok_matrix updates #7673 pull request

    if spalten > zeilen :
        raise Exception("You sha'll not get symmetrical context")
        #may need to fix this in the future due to other non symmetrical feature
    
    if spalten == zeilen:
        print('mirroring')
        #print(coocurrence.toarray())
        coocurrence = coocurrence + tril(coocurrence,k=-1).transpose()
    return coocurrence

In [4]:
import re
import os
import h5py
import numpy as np
import sys
sys.path.append('..')
from Vocabulary import *

# The default block length of 20000 must be divisible by this number. 
split_length = 4 #2,4,5,10
size = int(20000/split_length)

path = 'S:\\tmp\\coocurrence_blocks'
path_hdf = r'S:\\base_coocurrence_hdf_'+str(size)+'\\'
regex = r'block_([0-9]{1,})_([0-9]{1,})'

vocab = Vocabulary()
vocab.load('..\\vocabs\\baseline')
unvocab = Vocabulary()
unvocab.load('..\\vocabs\\unfilteredbaseline')


for file_name in os.listdir(path):
    match = re.match(regex, file_name)
    i,j   = match.group(1), match.group(2)
    print('output for progress ' + str(i)+','+str(j))
    if(i < j):
        continue
        
    
    co_occurence_dok = load_co_occurence(path,i,j)
    co_occurence_np  = co_occurence_dok.toarray()
    
    
    if(i == j):
        print('start diag')
        start_index = 20000 * int(i)
        for diagonal in range(20000):
            try:
                word = vocab.id2Word[start_index + diagonal]
                count = unvocab.word_frequency[word]
                co_occurence_np[diagonal,diagonal] = count
            except: 
                pass
        
        print('end diag')
    
    for sub_i in range(split_length):
        for sub_j in range(split_length):
            a,b = int(i)*split_length+sub_i , int(j)*split_length+sub_j
            filename = 'co_occurence_{a}_{b}_with_diag.hdf5'.format(a = a,b = b)
            hdf_file = h5py.File( path_hdf + filename, "w")
            co_occurence_hdf = hdf_file.create_dataset("co-ocurrence", (size, size))
            co_occurence_hdf[:,:] = co_occurence_np[sub_i*size:(sub_i+1)*size,sub_j*size:(sub_j+1)*size]
            hdf_file.close()
        

output for progress 0,0
mirroring


KeyboardInterrupt: 